In [5]:

import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC,LinearSVC
from sklearn.multiclass import OneVsRestClassifier
nPoints = 2000
data = np.zeros((4,nPoints))


In [4]:
#Tesseract
a = 100
nPoints = 32*a
data = np.zeros((4,nPoints))
b = np.arange(0,32*a)
b = b % a
b = b.dot(2/a) - 1
for i in range(8*a):
    data[0,i] = b[i]
    data[1,i] = (-1)**(i // (a) % 2)
    data[2,i] = (-1)**(i // (2*a) % 2)
    data[3,i] = (-1)**(i // (4*a) % 2)
for i in range(8*a,16*a):
    data[1,i] = b[i]
    data[2,i] = (-1)**(i // (a) % 2)
    data[3,i] = (-1)**(i // (2*a) % 2)
    data[0,i] = (-1)**(i // (4*a) % 2)
for i in range(16*a,24*a):
    data[2,i] = b[i]
    data[3,i] = (-1)**(i // (a) % 2)
    data[0,i] = (-1)**(i // (2*a) % 2)
    data[1,i] = (-1)**(i // (4*a) % 2)
for i in range(24*a,32*a):
    data[3,i] = b[i]
    data[0,i] = (-1)**(i // (a) % 2)
    data[1,i] = (-1)**(i // (2*a) % 2)
    data[2,i] = (-1)**(i // (4*a) % 2)

In [2]:
#Sphere
nPoints = 5000
data = np.zeros((3,nPoints))
b = 2*np.pi*np.random.uniform(0,1,2)
for i in range(0,nPoints):
    a = b*i
    data[0,i] = np.sin(a[0])*np.cos(a[1])
    data[1,i] = np.sin(a[0])*np.sin(a[1])
    data[2,i] = np.cos(a[0])

In [6]:
#Clifford Torus
b = 2*np.pi*np.random.uniform(0,1,2)
for i in range(0,nPoints):
    a = b*i
    data[:,i] = (np.cos(a[0]),np.sin(a[0]),np.cos(a[1]),np.sin(a[1]))

In [2]:
#Klein Bottle random distribution

r = 1
p = 1
e = 0.01
for i in range(0,nPoints):
    a = 2*np.pi*np.random.uniform(0,1,2)
    data[0,i] = r*(np.cos(a[0]/2)*np.cos(a[1]) - np.sin(a[0]/2)*np.sin(2*a[1]))
    data[1,i] = r*(np.sin(a[0]/2)*np.cos(a[1]) + np.cos(a[0]/2)*np.sin(2*a[1]))
    data[2,i] = p*np.cos(a[0])*(1 + e*np.sin(a[1]))
    data[3,i] = p*np.sin(a[0])*(1 + e*np.sin(a[1]))

In [3]:
#Klein Bottle
r = 1
p = 1
e = 0.01
b = 2*np.pi*np.random.uniform(0,1,2)*0.1
for i in range(0,nPoints):
    a = b*i
    data[0,i] = r*(np.cos(a[0]/2)*np.cos(a[1]) - np.sin(a[0]/2)*np.sin(2*a[1]))
    data[1,i] = r*(np.sin(a[0]/2)*np.cos(a[1]) + np.cos(a[0]/2)*np.sin(2*a[1]))
    data[2,i] = p*np.cos(a[0])*(1 + e*np.sin(a[1]))
    data[3,i] = p*np.sin(a[0])*(1 + e*np.sin(a[1]))

In [7]:
#VARIABLES
nSteps = 1000
stepSize =0.05
def getAlpha(d,stepSize):
    """
    NEEDS IMPLEMENTATION
    Should produce 1xd(d-1)/2 array of position in grand tour.

    """
    p = d*(d-1)/2     
    primeList = []
    count = 1
    while len(primeList) < p:
        count += 1
        primeBool = False
        for i in range(2, count - 1):
            if count % i == 0:
                primeBool = True
        if primeBool == False:
            irrational = (np.sqrt(count)%1)
            primeList.append(irrational)

    primeList = np.asarray(primeList)
    primeList = primeList.dot(stepSize)
    """
    Irrational number generation using exponentials, not being used
    p = int(d*(d-1)/2)
    alpha = np.zeros(p) #alpha(t) parameters defining grand tour in G2,d

    for i in range(0,p):
        alpha[i] = (np.exp(i) % 1) * 2 * np.pi
        
    alpha = alpha.dot(0.001)
    """
    
    return primeList


def getAngles(alpha,d):
    """""
    Inputs: 
    alpha = 1xd(d-1)/2 array defining position on grand tour
    d = dimensions of data
    Outputs a dxd array of angles required for the transformation
    """
    theta = np.zeros((d,d));
    i = 0;
    k = 0;
    
    while i < d-1:
        j = i + 1;
        
        while j < d:
            theta[i][j] = alpha[k];
            j += 1;
            k += 1;
    
        i+= 1;
        
    return theta;


def RotationMatrix(i, j, d, theta):
    """
    Inputs:
    i = first indicie of rotating plane
    j = second indicie of rotating plane
    d = dimensions of data
    theta = dxd array of angle of rotation of rotating plane

    Outputs a rotating matrix to rotate plane of ixj plane by theta_ij
    """
    R = np.identity(d)
    R[i,i] = np.cos(theta)
    R[i,j] = -1*np.sin(theta)
    R[j,i] = np.sin(theta)
    R[j,j] = np.cos(theta)
    return R


def BetaFn(d, theta):
    """
    Inputs:
    d = dimensions of data
    theta = dxd array of angle of rotation ixj plane

    Outputs the full matrix transformation for all rotations
    """
    b = RotationMatrix(1, 2, d, theta[1,2])
    i = 1
    j = 2
    for i in range(d):
        for j in range(d):
            if j <= i:
                continue
            if i==1 and j==2:
                continue
            b = np.matmul(b, RotationMatrix(i, j, d, theta[i,j]))
            
    return b


def GrandTour(data, nSteps,stepSize):
    """
    Inputs:
    data = array of data points, dimensions x npoints
    Outputs a 3D array number of points x t x dimensions, where t
    the time step at that point in the tour
    """

    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    tData = np.zeros((nSteps,d,nPoints)) #initialise 3d matrix to store stransforemd data at each timestep
    tBeta = np.zeros((nSteps,d,d))
    Alpha = getAlpha(d,stepSize)

    
    for t in range(0, nSteps):
        
        
        alpha = Alpha.dot(t)
        theta = getAngles(alpha, d)
        b = BetaFn(d, theta)
        a = np.matmul(b, data)
        tData[t,:,:] = a
        tBeta[t,:,:] = b
        
    return tData

def Transform(data,alpha):
    d = np.shape(data)[0]
    theta = getAngles(alpha,d)
    b = BetaFn(d,theta)
    a = np.matmul(b,data)
    return a


def singleAlphaRotation(data,nSteps):
    """
    Alters a single parameter in the alpha vector 
    does a 2pi rotation
    """
    d = np.shape(data)[0] #dimensions of data
    nPoints = np.shape(data)[1] #number of data points
    A = np.zeros((nSteps,d,nPoints))
    alpha = getAlpha(d, 1)
    print(alpha)
    dAlpha = alpha.dot(0)
    p = int(d*(d-1)/2)
    tData = np.zeros((0,d,nPoints))
    
    
    for j in range(0,p):
        
        dAlpha = alpha.dot(0)
        dAlpha[j] = np.pi * 2 /nSteps

        for i in range(0,100):
            alpha = alpha + dAlpha
            a = Transform(data,alpha)
            A[i] = a
            i += 1
        tData = np.append(tData,A,axis = 0)
        j+=1
    
    return tData

tData = GrandTour(data,nSteps,stepSize)


In [ ]:
from pyqtgraph.Qt import QtGui, QtCore
import pandas as pd
import pyqtgraph as pg
import time


app = QtGui.QApplication([])
mw = QtGui.QMainWindow()
mw.resize(800,800)
view = pg.GraphicsLayoutWidget()  ## GraphicsView with GraphicsLayout inserted by default
mw.setCentralWidget(view)
mw.show()
mw.setWindowTitle('pyqtgraph example: ScatterPlot')

## create four areas to add plots
w1 = view.addPlot()


## There are a few different ways we can draw scatter plots; each is optimized for different types of data:


## 1) All spots identical and transform-invariant (top-left plot). 
## In this case we can get a huge performance boost by pre-rendering the spot 
## image and just drawing that image repeatedly.
nPoints = np.shape(tData)[2]
s1 = pg.ScatterPlotItem(size=10, pen=pg.mkPen(None), brush=pg.mkBrush(0, 227, 17,120))
pos = np.zeros((2,nPoints))
j = 0
pos[0,:] = tData[j,0,:]
pos[1,:] = tData[j,1,:]
spots = [{'pos': pos[:,i], 'data': 1} for i in range(nPoints)]
s1.addPoints(spots)
w1.addItem(s1)

def update():
    global j
    j += 1
    if j > np.shape(tData)[0]-1:
        j = 0
    pos[0,:] = tData[j,0,:]
    pos[1,:] = tData[j,1,:]
    spots = [{'pos': pos[:,i], 'data': 1} for i in range(nPoints)]
    s1.setData(spots)

t = QtCore.QTimer()
t.timeout.connect(update)
t.start(10)

## 2) Spots are transform-invariant, but not identical (top-right plot). 
## In this case, drawing is almsot as fast as 1), but there is more startup 
## overhead and memory usage since each spot generates its own pre-rendered 
## image.



## Start Qt event loop unless running in interactive mode.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()



In [9]:
!pip install pyqtgraph

    100% |████████████████████████████████| 1.6MB 9.1MB/s eta 0:00:01
  Running setup.py bdist_wheel for pyqtgraph ... done
  Stored in directory: /Users/karanmukhi/Library/Caches/pip/wheels/93/a2/de/184ecdefdb91d760706d02389015dfa3b96e927eb1c42ace37
Successfully built pyqtgraph


In [14]:
p = d*(d-1)/2     
primeList = []
count = 1
while len(primeList) < p:
    count += 1
    primeBool = False
    for i in range(2, count - 1):
        if count % i == 0:
            primeBool = True
    if primeBool == False:
        irrational = (np.sqrt(count)%1)
        primeList.append(irrational)

primeList = np.asarray(primeList)
print(primeList)
primeList = primeList.dot(stepSize)
print(primeList)

[0.41421356 0.73205081 0.23606798]
[4.14213562 7.32050808 2.36067977]


In [69]:
b


array([-1.  , -0.96, -0.92, ...,  0.88,  0.92,  0.96])

In [61]:
(-1)**0

1

In [45]:
2%1

0